In [2]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
import traceback


In [3]:
# Load environment variables
load_dotenv()
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")




In [7]:

from langchain_huggingface import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=0.5,
    max_new_tokens=1024,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)


In [8]:
# Sample JSON format for MCQs
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}




In [ ]:
# Prompt for quiz generation
TEMPLATE = """
Text: {text}

You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in a {tone} tone. 

Make sure the questions are not repeated and all questions are directly based on the text.
Ensure the format matches the RESPONSE_JSON below. Create exactly {number} MCQs.

### RESPONSE_JSON
{response_json}
"""


quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)


quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


# Prompt for evaluating the quiz quality
TEMPLATE2 = """
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students,\
you need to evaluate the complexity of the quiz and provide a short 50-word analysis.

If the quiz is too complex or not well-suited for the students, update the questions and modify the tone accordingly.

Quiz_MCQs:
{quiz}

Provide your expert review of the quiz:
"""


quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)


review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


# Combined chain
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)



# Read input content
file_path = r"C:\Users\sridh\mcqgen\data.txt"
with open(file_path, 'r') as file:
    TEXT = file.read()


# Configuration
NUMBER = 5
SUBJECT = "biology"
TONE = "simple"
# Run the chain
response = generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)


# Extract quiz and convert to table
quiz = response.get("quiz")
try:
    quiz = json.loads(quiz)
except Exception as e:
    print("Failed to parse quiz JSON:", e)
    print("Raw quiz response:\n", quiz)
    quiz = {}

quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [f"{option}: {option_value}" for option, option_value in value["options"].items()]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})


# Save to CSV
quiz_df = pd.DataFrame(quiz_table_data)
quiz_df.to_csv("mcq_output.csv", index=False)


# Print review
print("\nQuiz Review:\n", response.get("review"))